<a href="https://colab.research.google.com/github/py811/RLSS2022_AI_and_COVID/blob/main/Season_Correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing Libraries and the Covid Data 

In [ ]:
#importing useful libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.stats.mstats import kendalltau, spearmanr
from datetime import datetime, date, timedelta
import math
#import statsmodels.api as sm

import plotly.graph_objects as go
from plotly.subplots import make_subplots

#import pingouin as pg
import warnings
warnings.filterwarnings('ignore')


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#read COVID-19 data 
Covid_Data = pd.read_csv('https://raw.githubusercontent.com/py811/RLSS2022_AI_and_COVID/main/COVID_DATA.csv',usecols=[0,2,3,4,8,17,18,19,20,21])
#Covid_Data = pd.read_csv('merged_df.csv',usecols=[0,2,3,4,8,17,18,19,20,21])
Covid_Data.rename(columns={'Difference_Yesterday':'PCR_Postive_Count','Hospitalized_diff':'PCR_Positive_Hospitalised','Death_diff':'Deceased'}, inplace = True)
#data.drop([0,1], axis=0, inplace=True)
Covid_Data['PCR_Positive_Hospitalised'] = Covid_Data['PCR_Positive_Hospitalised'].fillna(0).astype('int64')
#Covid_Data

In [ ]:
Covid_Data1 = Covid_Data
Covid_Data1['Date']=pd.to_datetime(Covid_Data1['Date'].astype(str), format='%d-%m-%Y')

In [ ]:
#Aggregating the Covid Data for the whole state RLP
Covid_Data_avg = Covid_Data1.groupby(['Date']).agg({'PCR_Postive_Count' : 'sum', 'PCR_Positive_Hospitalised' : 'sum',
                                                     'Deceased' : 'sum','Total_Confirmed' : 'sum','Current_Infected' : 'sum',
                                             'temperature' : 'mean', 'wind_speed' : 'mean', 'relative_humidity': 'mean',
                                            }).reset_index()
#Covid_Data_avg.to_csv('Covid_Data_avg.csv')
Covid_Data_avg['Day_of_week'] = Covid_Data_avg['Date'].dt.day_name()
#Covid_Data_avg

### Adding Lag Columns 

In [ ]:
def lagged_col_rev(df, col, lag, fill_value=0):
    df[col+'_lag_-'+str(lag)] = df[col].shift(periods=-lag)
    return df

Covid_Data_avg_lag = Covid_Data_avg.copy()
lags = 7 

for lag in range(1,lags+1):
    Covid_avg_PCR_lag = lagged_col_rev(Covid_Data_avg_lag, 'PCR_Postive_Count', lag, fill_value=0)
    
for lag in range(1,lags+1):
    Covid_avg_CrIn_lag = lagged_col_rev(Covid_Data_avg_lag, 'Current_Infected', lag, fill_value=0)
    
Covid_avg_PCR_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/complete_data_lag/Covid_avg_PCR_lag.csv')
Covid_avg_CrIn_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/complete_data_lag/Covid_avg_CrIn_lag.csv')

### Defining Functions

In [ ]:
def find_cor_sig(data,final_list,season_name, covid_param, weather_param):
   # covid_param = ['PCR_Postive_Count','PCR_Positive_Hospitalised','Deceased']
   # weather_param = ['temperature','wind_speed','relative_humidity']
    return_list = []
    for i in range(0,len(covid_param)):
        for j in range(0,len(weather_param)):
            corr_kendall= kendalltau(data[covid_param[i]],data[weather_param[j]])
            corr = round(corr_kendall[0],4)
            sig = corr_kendall[1] <= 0.05
            return_list.append(('Kendall-Tau',season_name,covid_param[i],weather_param[j],corr,corr_kendall[1],sig))
    for i in range(0,len(covid_param)):
        for j in range(0,len(weather_param)):
            corr_spear= spearmanr(data[covid_param[i]],data[weather_param[j]])
            corr = round(corr_spear[0],4)
            sig = corr_spear[1] <= 0.05
            return_list.append(('Spearmann',season_name,covid_param[i],weather_param[j],corr,corr_spear[1],sig))        
        
    #corr_kendall1= data.corr(method='kendall')
    for val in range(0,len(return_list)):
        final_list.append(return_list[val])
    return final_list

def find_mul_cor(AB,CB,AC, n):
    value =((AB**2) + (CB**2) - (2*AB*CB*AC))/(1 -(AC**2))

    r = math.sqrt(value)

    r_squared = r**2

    correlation_C_WRT_AB = (1- (((1-r_squared)*(n-1))/(n-2-1)))

    return round(correlation_C_WRT_AB,4)

In [ ]:
#multiple correlation
def mul_corr_calc(corr_df,data_df,season,weather_param,n):
    mul_corr_list = []
    #-----------fetching rows only with significance value=TRUE--------------
    mul_corr_data = corr_df.loc[(corr_df["Significance"]==True)]

    #-----correlation between temperature and relative humidity--------------
    corr_kendall= kendalltau(data_df['temperature'],data_df['relative_humidity'])
    corr_TR_ken = round(corr_kendall[0],4)

    corr_spear= spearmanr(data_df['temperature'],data_df['relative_humidity'])
    corr_TR_spe = round(corr_spear[0],4)
    
    covid_param =  ['PCR_Postive_Count','PCR_Positive_Hospitalised','Deceased', 'Current_Infected', 'Total_Confirmed']
    correlation = ['Kendall-Tau','Spearmann']
    corr_mul_list=[]

    for i in correlation:
        for j in covid_param:
            result_list=[]
            for k in weather_param:
                temp = mul_corr_data.loc[(mul_corr_data["Covid_Column"]== j) & (mul_corr_data['CorrelationMethod']==i) & (mul_corr_data['Weather_Column']==k)]
                new_data = []
                new_data = temp['Correlation'].array
                if (len(new_data)>0):
                    result_list.append(new_data[0])
            if(len(result_list)==2):
                if(i=='Kendall-Tau'):
                    result = find_mul_cor(corr_TR_ken,result_list[0],result_list[1], n)
                    corr_mul_list.append(('Kendall-Tau',season,j,result_list[0],result_list[1],result))
                else:
                    result = find_mul_cor(corr_TR_spe,result_list[0],result_list[1], n)
                    corr_mul_list.append(('Spearmann',season,j,result_list[0],result_list[1],result))
    return corr_mul_list 

### Autumn 

In [ ]:
#Autumn
str_dt_aut = '31-08-2021'
end_dt_aut = '30-11-2021'
data_autumn = Covid_Data_avg.loc[(Covid_Data_avg['Date'] >= pd.to_datetime(str_dt_aut, format='%d-%m-%Y')) & (Covid_Data_avg['Date'] <= pd.to_datetime(end_dt_aut, format='%d-%m-%Y'))]

lags = 7 

for lag in range(1,lags+1):
    data_autumn_PCR_lag = lagged_col_rev(data_autumn, 'PCR_Postive_Count', lag, fill_value=0)
    
for lag in range(1,lags+1):
    data_autumn_CrIn_lag = lagged_col_rev(data_autumn, 'Current_Infected', lag, fill_value=0)

data_autumn_PCR_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/Autumn/data_autumn_PCR_lag.csv')
data_autumn_CrIn_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/Autumn/data_autumn_CrIn_lag.csv')

data = data_autumn
print(data[["temperature","relative_humidity"]].mean())

corr_kendall= kendalltau(data['temperature'],data['relative_humidity'])
print('Kendall-Tau Correlation between Temperature and Relative Humidity is',round(corr_kendall[0],4), 'for p-value', corr_kendall[1], 'and hence significance as',corr_kendall[1] <= 0.05)

corr_spear= spearmanr(data['temperature'],data['relative_humidity'])
print('Spearmann Correlation between Temperature and Relative Humidity is',round(corr_spear[0],4), 'for p-value', corr_spear[1], 'and hence significance as',corr_spear[1] <= 0.05) 


temperature          10.114351
relative_humidity    83.803754
dtype: float64
Kendall-Tau Correlation between Temperature and Relative Humidity is -0.5002 for p-value 1.6336192080365082e-12 and hence significance as True
Spearmann Correlation between Temperature and Relative Humidity is -0.6827 for p-value 6.623248910183961e-14 and hence significance as True


In [ ]:
data = data_autumn.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="Humidity", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside")

# Set y-axes titles
fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue', range = [-5,30])

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered', range = [30,100])

fig.update_layout(plot_bgcolor="white")

fig.show()

#fig.savefig("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Autumn/Avg_Temp_Hum.png")

In [ ]:
data = data_autumn.copy()
covid_param = ['PCR_Postive_Count','PCR_Positive_Hospitalised','Deceased', 'Current_Infected', 'Total_Confirmed']
weather_param = ['temperature','wind_speed','relative_humidity']
final_list =[]
final_list = find_cor_sig(data,final_list,'Autumn',covid_param,weather_param)
corr_data = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_data.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Autumn/Correlation.csv')
corr_data

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Autumn,PCR_Postive_Count,temperature,-0.2884,4.680361e-05,True
1,Kendall-Tau,Autumn,PCR_Postive_Count,wind_speed,0.0222,7.537672e-01,False
2,Kendall-Tau,Autumn,PCR_Postive_Count,relative_humidity,0.1580,2.578334e-02,True
3,Kendall-Tau,Autumn,PCR_Positive_Hospitalised,temperature,-0.2086,3.508869e-03,True
4,Kendall-Tau,Autumn,PCR_Positive_Hospitalised,wind_speed,0.0258,7.180186e-01,False
5,Kendall-Tau,Autumn,PCR_Positive_Hospitalised,relative_humidity,0.0967,1.759560e-01,False
6,Kendall-Tau,Autumn,Deceased,temperature,-0.3282,9.956353e-06,True
7,Kendall-Tau,Autumn,Deceased,wind_speed,-0.0111,8.809486e-01,False
8,Kendall-Tau,Autumn,Deceased,relative_humidity,0.2736,2.305951e-04,True
9,Kendall-Tau,Autumn,Current_Infected,temperature,-0.1921,6.693265e-03,True


In [ ]:
corr_data_pos = corr_data.loc[(corr_data["Significance"]==True)]
#corr_data_pos
corr_data_pos

#Overall humidity has a very negligible correlation with daily count
#Temperature has a very negligible correlation with daily count but better than relative humidity

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Autumn,PCR_Postive_Count,temperature,-0.2884,4.680361e-05,True
2,Kendall-Tau,Autumn,PCR_Postive_Count,relative_humidity,0.1580,2.578334e-02,True
3,Kendall-Tau,Autumn,PCR_Positive_Hospitalised,temperature,-0.2086,3.508869e-03,True
6,Kendall-Tau,Autumn,Deceased,temperature,-0.3282,9.956353e-06,True
8,Kendall-Tau,Autumn,Deceased,relative_humidity,0.2736,2.305951e-04,True
9,Kendall-Tau,Autumn,Current_Infected,temperature,-0.1921,6.693265e-03,True
12,Kendall-Tau,Autumn,Total_Confirmed,temperature,-0.7649,3.452948e-27,True
14,Kendall-Tau,Autumn,Total_Confirmed,relative_humidity,0.4974,2.184453e-12,True
15,Spearmann,Autumn,PCR_Postive_Count,temperature,-0.4444,9.074307e-06,True
17,Spearmann,Autumn,PCR_Postive_Count,relative_humidity,0.2583,1.291950e-02,True


#### Doing Correlation with Temperature

In [ ]:
corr_data_pos.loc[(corr_data_pos["Weather_Column"]=='temperature')]

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Autumn,PCR_Postive_Count,temperature,-0.2884,4.680361e-05,True
3,Kendall-Tau,Autumn,PCR_Positive_Hospitalised,temperature,-0.2086,3.508869e-03,True
6,Kendall-Tau,Autumn,Deceased,temperature,-0.3282,9.956353e-06,True
9,Kendall-Tau,Autumn,Current_Infected,temperature,-0.1921,6.693265e-03,True
12,Kendall-Tau,Autumn,Total_Confirmed,temperature,-0.7649,3.452948e-27,True
15,Spearmann,Autumn,PCR_Postive_Count,temperature,-0.4444,9.074307e-06,True
18,Spearmann,Autumn,PCR_Positive_Hospitalised,temperature,-0.3080,2.817824e-03,True
21,Spearmann,Autumn,Deceased,temperature,-0.4421,1.022659e-05,True
24,Spearmann,Autumn,Current_Infected,temperature,-0.3334,1.165193e-03,True
27,Spearmann,Autumn,Total_Confirmed,temperature,-0.9263,6.327653e-40,True


In [ ]:
#Multiple Correlation
mul_list = []
weather_param = ['temperature','relative_humidity']
mul_list = mul_corr_calc(corr_data,data_autumn,'Autumn',weather_param,92)
corr_mul_aut_TH = pd.DataFrame (mul_list, columns = ['CorrelationMethod','Season','Covid_Column','Temperature','Relative_Humidity','Multiple_Correlation'])
corr_mul_aut_TH.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Autumn/Multiple_Correlation.csv')
corr_mul_aut_TH

,CorrelationMethod,Season,Covid_Column,Temperature,Relative_Humidity,Multiple_Correlation
0,Kendall-Tau,Autumn,PCR_Postive_Count,-0.2884,0.1580,0.2793
1,Kendall-Tau,Autumn,Deceased,-0.3282,0.2736,0.2738
2,Kendall-Tau,Autumn,Total_Confirmed,-0.7649,0.4974,0.5952
3,Spearmann,Autumn,PCR_Postive_Count,-0.4444,0.2583,0.5328
4,Spearmann,Autumn,Deceased,-0.4421,0.3768,0.4948
5,Spearmann,Autumn,Total_Confirmed,-0.9263,0.7082,0.8563


In [ ]:
data = data_autumn.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="Daily Positive Count", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Daily Positive Count", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green', range = [0,3000])

fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue', range = [0,30])

fig.update_layout(plot_bgcolor="white")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    bordercolor="Black",
    borderwidth=1
))

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Autumn/PCR_Temp.png")

In [ ]:
data = data_autumn.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['Current_Infected'],name="Current Infected", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Current Infected", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Autumn/CrIn_Temp.png")

In [ ]:
#correlation for lagged data with Current Infected
data = data_autumn_CrIn_lag.copy()
covid_param = ['Current_Infected','Current_Infected_lag_-1', 'Current_Infected_lag_-2','Current_Infected_lag_-3', 
               'Current_Infected_lag_-4','Current_Infected_lag_-5', 'Current_Infected_lag_-6',
               'Current_Infected_lag_-7']
weather_param = ['temperature']
final_list =[]
final_list = find_cor_sig(data,final_list,'Autumn',covid_param,weather_param)

corr_data_temp = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
#corr_data_temp.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Autumn/CrIn_lag_corr.csv')
corr_data_temp
#no concrete lag correlation

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Autumn,Current_Infected,temperature,-0.1921,0.006693,True
1,Kendall-Tau,Autumn,Current_Infected_lag_-1,temperature,-0.1873,0.008187,True
2,Kendall-Tau,Autumn,Current_Infected_lag_-2,temperature,-0.1758,0.013051,True
3,Kendall-Tau,Autumn,Current_Infected_lag_-3,temperature,-0.1782,0.011866,True
4,Kendall-Tau,Autumn,Current_Infected_lag_-4,temperature,-0.1892,0.007557,True
5,Kendall-Tau,Autumn,Current_Infected_lag_-5,temperature,-0.1978,0.005227,True
6,Kendall-Tau,Autumn,Current_Infected_lag_-6,temperature,-0.2236,0.001594,True
7,Kendall-Tau,Autumn,Current_Infected_lag_-7,temperature,-0.2379,0.000781,True
8,Spearmann,Autumn,Current_Infected,temperature,-0.3334,0.001165,True
9,Spearmann,Autumn,Current_Infected_lag_-1,temperature,-0.3395,0.000930,True


In [ ]:
#with PCR lagged Data
data = data_autumn_PCR_lag.copy()
covid_param = ['PCR_Postive_Count','PCR_Postive_Count_lag_-1', 'PCR_Postive_Count_lag_-2','PCR_Postive_Count_lag_-3', 
               'PCR_Postive_Count_lag_-4','PCR_Postive_Count_lag_-5', 'PCR_Postive_Count_lag_-6',
               'PCR_Postive_Count_lag_-7']
weather_param = ['temperature']
final_list =[]
final_list = find_cor_sig(data,final_list,'Autumn',covid_param,weather_param)

corr_data_temp = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_data_temp
#corr_data_temp.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Autumn/PCR_lag_corr.csv')
#no concrete lag correlation

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Autumn,PCR_Postive_Count,temperature,-0.2884,4.680361e-05,True
1,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-1,temperature,-0.3100,1.216566e-05,True
2,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-2,temperature,-0.3080,1.376260e-05,True
3,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-3,temperature,-0.3276,3.760663e-06,True
4,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-4,temperature,-0.3568,4.761602e-07,True
5,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-5,temperature,-0.3907,3.495264e-08,True
6,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-6,temperature,-0.4299,1.296962e-09,True
7,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-7,temperature,-0.4471,2.779222e-10,True
8,Spearmann,Autumn,PCR_Postive_Count,temperature,-0.4444,9.074307e-06,True
9,Spearmann,Autumn,PCR_Postive_Count_lag_-1,temperature,-0.4584,4.317277e-06,True


In [ ]:
data = data_autumn_PCR_lag.copy()
#print(data)
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="PCR_Postive_Count", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count_lag_-3'],name="PCR_Postive_Count lag 3", line=dict(color='green',
                              dash='dot')),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="temperature", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Autumn/Lag_PCR.png")

Subdiving into 2 parts

In [ ]:
data_autumn_PCR_lag

,Date,PCR_Postive_Count,PCR_Positive_Hospitalised,Deceased,Total_Confirmed,Current_Infected,temperature,wind_speed,relative_humidity,Day_of_week,...,PCR_Postive_Count_lag_-5,PCR_Postive_Count_lag_-6,PCR_Postive_Count_lag_-7,Current_Infected_lag_-1,Current_Infected_lag_-2,Current_Infected_lag_-3,Current_Infected_lag_-4,Current_Infected_lag_-5,Current_Infected_lag_-6,Current_Infected_lag_-7
29,2021-08-31,588,31,3,146163,6267,15.206267,11.380667,80.758667,Tuesday,...,393.0,532.0,668.0,6929.0,7535.0,7916.0,8013.0,8187.0,8478.0,8876.0
30,2021-09-01,815,26,5,146978,6929,14.883333,11.327067,80.010667,Wednesday,...,532.0,668.0,742.0,7535.0,7916.0,8013.0,8187.0,8478.0,8876.0,9388.0
31,2021-09-02,786,21,-1,147764,7535,16.131067,12.135067,74.312000,Thursday,...,668.0,742.0,759.0,7916.0,8013.0,8187.0,8478.0,8876.0,9388.0,9868.0
32,2021-09-03,593,19,0,148357,7916,17.460000,9.000667,70.646667,Friday,...,742.0,759.0,559.0,8013.0,8187.0,8478.0,8876.0,9388.0,9868.0,10125.0
33,2021-09-04,326,5,1,148683,8013,17.987600,7.665467,70.237333,Saturday,...,759.0,559.0,284.0,8187.0,8478.0,8876.0,9388.0,9868.0,10125.0,10038.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,2021-11-26,2212,45,4,199611,24344,1.887200,15.794400,86.692000,Friday,...,NaN,NaN,NaN,24043.0,24104.0,25312.0,26524.0,NaN,NaN,NaN
117,2021-11-27,595,11,2,200206,24043,1.940933,14.093733,87.816000,Saturday,...,NaN,NaN,NaN,24104.0,25312.0,26524.0,NaN,NaN,NaN,NaN
118,2021-11-28,925,27,3,201131,24104,1.455467,8.472400,90.604000,Sunday,...,NaN,NaN,NaN,25312.0,26524.0,NaN,NaN,NaN,NaN,NaN
119,2021-11-29,2035,47,14,203166,25312,1.293067,14.578086,86.973333,Monday,...,NaN,NaN,NaN,26524.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
Covid_avg_PCR_lag

,Date,PCR_Postive_Count,PCR_Positive_Hospitalised,Deceased,Total_Confirmed,Current_Infected,temperature,wind_speed,relative_humidity,Day_of_week,...,PCR_Postive_Count_lag_-5,PCR_Postive_Count_lag_-6,PCR_Postive_Count_lag_-7,Current_Infected_lag_-1,Current_Infected_lag_-2,Current_Infected_lag_-3,Current_Infected_lag_-4,Current_Infected_lag_-5,Current_Infected_lag_-6,Current_Infected_lag_-7
0,2021-08-02,122,3,0,138639,1560,15.408933,10.650933,83.221333,Monday,...,51.0,47.0,115.0,1646.0,1758.0,1830.0,1901.0,1875.0,1834.0,1870.0
1,2021-08-03,142,8,0,138781,1646,14.946933,10.774667,87.574667,Tuesday,...,47.0,115.0,161.0,1758.0,1830.0,1901.0,1875.0,1834.0,1870.0,1959.0
2,2021-08-04,170,6,1,138951,1758,16.023067,7.270933,81.329333,Wednesday,...,115.0,161.0,234.0,1830.0,1901.0,1875.0,1834.0,1870.0,1959.0,2138.0
3,2021-08-05,129,1,0,139080,1830,17.316933,9.500400,80.693333,Thursday,...,161.0,234.0,247.0,1901.0,1875.0,1834.0,1870.0,1959.0,2138.0,2291.0
4,2021-08-06,156,10,3,139236,1901,16.960667,16.658667,80.053333,Friday,...,234.0,247.0,228.0,1875.0,1834.0,1870.0,1959.0,2138.0,2291.0,2440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,2022-07-25,6598,63,14,1184097,84698,24.600400,15.073067,51.744000,Monday,...,NaN,NaN,NaN,89989.0,94494.0,92589.0,90382.0,NaN,NaN,NaN
329,2022-07-26,5792,137,6,1189889,89989,20.124400,12.751867,58.258667,Tuesday,...,NaN,NaN,NaN,94494.0,92589.0,90382.0,NaN,NaN,NaN,NaN
330,2022-07-27,4841,104,3,1194730,94494,18.504533,12.537733,54.682667,Wednesday,...,NaN,NaN,NaN,92589.0,90382.0,NaN,NaN,NaN,NaN,NaN
331,2022-07-28,3913,76,7,1198643,92589,20.134000,12.251200,49.390667,Thursday,...,NaN,NaN,NaN,90382.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
str_dt_aut = '31-08-2021'
end_dt_aut = '14-10-2021'
data_autumn_1 = data_autumn_PCR_lag.loc[(data_autumn_PCR_lag['Date'] >= pd.to_datetime(str_dt_aut, format='%d-%m-%Y')) & (data_autumn_PCR_lag['Date'] <= pd.to_datetime(end_dt_aut, format='%d-%m-%Y'))]

data = data_autumn_1.copy()
covid_param = ['PCR_Postive_Count','PCR_Postive_Count_lag_-1', 'PCR_Postive_Count_lag_-2','PCR_Postive_Count_lag_-3', 
               'PCR_Postive_Count_lag_-4','PCR_Postive_Count_lag_-5', 'PCR_Postive_Count_lag_-6',
               'PCR_Postive_Count_lag_-7']
weather_param = ['temperature']
final_list =[]
final_list = find_cor_sig(data,final_list,'Autumn',covid_param,weather_param)

corr_data_temp = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_data_temp


,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Autumn,PCR_Postive_Count,temperature,0.2597,0.011931,True
1,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-1,temperature,0.2496,0.015677,True
2,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-2,temperature,0.3103,0.002671,True
3,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-3,temperature,0.4053,0.000087,True
4,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-4,temperature,0.3547,0.000595,True
5,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-5,temperature,0.2436,0.018391,True
6,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-6,temperature,0.1304,0.206956,False
7,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-7,temperature,0.0495,0.631685,False
8,Spearmann,Autumn,PCR_Postive_Count,temperature,0.3739,0.011396,True
9,Spearmann,Autumn,PCR_Postive_Count_lag_-1,temperature,0.3574,0.015945,True


In [ ]:
data = data_autumn_1.copy()
#print(data)
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="PCR_Postive_Count", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count_lag_-3'],name="PCR_Postive_Count lag 3", line=dict(color='green',
                              dash='dot')),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="temperature", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

fig.show()

In [ ]:
str_dt_aut = '15-10-2021'
end_dt_aut = '30-11-2021'
data_autumn_2 = data_autumn_PCR_lag.loc[(data_autumn_PCR_lag['Date'] >= pd.to_datetime(str_dt_aut, format='%d-%m-%Y')) & (data_autumn_PCR_lag['Date'] <= pd.to_datetime(end_dt_aut, format='%d-%m-%Y'))]

data = data_autumn_2.copy()
covid_param = ['PCR_Postive_Count','PCR_Postive_Count_lag_-1', 'PCR_Postive_Count_lag_-2','PCR_Postive_Count_lag_-3', 
               'PCR_Postive_Count_lag_-4','PCR_Postive_Count_lag_-5', 'PCR_Postive_Count_lag_-6',
               'PCR_Postive_Count_lag_-7']
weather_param = ['temperature']
final_list =[]
final_list = find_cor_sig(data,final_list,'Autumn',covid_param,weather_param)

corr_data_temp = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_data_temp

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Autumn,PCR_Postive_Count,temperature,-0.4553,6.420319e-06,True
1,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-1,temperature,-0.4535,6.999255e-06,True
2,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-2,temperature,-0.4091,5.045465e-05,True
3,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-3,temperature,-0.4331,1.771087e-05,True
4,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-4,temperature,-0.4331,1.771087e-05,True
5,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-5,temperature,-0.4553,6.420319e-06,True
6,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-6,temperature,-0.5035,6.071261e-07,True
7,Kendall-Tau,Autumn,PCR_Postive_Count_lag_-7,temperature,-0.4868,1.408053e-06,True
8,Spearmann,Autumn,PCR_Postive_Count,temperature,-0.6523,6.791849e-07,True
9,Spearmann,Autumn,PCR_Postive_Count_lag_-1,temperature,-0.6194,3.451258e-06,True


### plotting it for the 14 days period with lag 

In [ ]:
#lowest value weeks in all seasons
#Autumn: 22.11.2021 to 05.12.2021
low_aut = Covid_Data_avg.loc[(Covid_Data_avg['Date'] >= pd.to_datetime('22-11-2021', format='%d-%m-%Y')) & (Covid_Data_avg['Date'] <= pd.to_datetime('05-12-2021', format='%d-%m-%Y'))]
low_aut= low_aut.reset_index()
#low_data_aut
lags = 7 

for lag in range(1,lags+1):
    low_aut_PCR_lag = lagged_col_rev(low_aut, 'PCR_Postive_Count', lag, fill_value=0)
    
for lag in range(1,lags+1):
    low_aut_CrIn_lag = lagged_col_rev(low_aut, 'Current_Infected', lag, fill_value=0)

low_aut_PCR_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/Autumn/Lowest_week_PCR_lag.csv')
low_aut_CrIn_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/Autumn/Lowest_week_CrIn_lag.csv')

In [ ]:
data = low_aut.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="Humidity", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside")

# Set y-axes titles
fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue', range = [-5,30])

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered', range = [30,100])

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Autumn/Low_week_Avg_Temp_Hum.png")

In [ ]:
data = low_aut.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="Daily Positive Count", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Daily Positive Count", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Autumn/Low_Week_PCR_Temp.png")

In [ ]:
data = low_aut.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['Current_Infected'],name="Current Infected", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Current Infected", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Autumn/Low_Week_CrIn_Temp.png")

In [ ]:
#correlation for lagged data with PCR count
data = low_aut_PCR_lag.copy()
covid_param = ['PCR_Postive_Count','PCR_Postive_Count_lag_-1', 'PCR_Postive_Count_lag_-2','PCR_Postive_Count_lag_-3', 
               'PCR_Postive_Count_lag_-4','PCR_Postive_Count_lag_-5', 'PCR_Postive_Count_lag_-6',
               'PCR_Postive_Count_lag_-7']
weather_param = ['temperature']
final_list =[]
final_list = find_cor_sig(data,final_list,'Lowest_Autumn_week',covid_param,weather_param)

corr_aut_low_pcr = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_aut_low_pcr.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Autumn/Low_Week_PCR_lag_Corr.csv')
corr_aut_low_pcr

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Lowest_Autumn_week,PCR_Postive_Count,temperature,-0.0769,0.747189,False
1,Kendall-Tau,Lowest_Autumn_week,PCR_Postive_Count_lag_-1,temperature,0.2308,0.279180,False
2,Kendall-Tau,Lowest_Autumn_week,PCR_Postive_Count_lag_-2,temperature,0.4066,0.047176,True
3,Kendall-Tau,Lowest_Autumn_week,PCR_Postive_Count_lag_-3,temperature,-0.2308,0.279180,False
4,Kendall-Tau,Lowest_Autumn_week,PCR_Postive_Count_lag_-4,temperature,-0.2308,0.279180,False
5,Kendall-Tau,Lowest_Autumn_week,PCR_Postive_Count_lag_-5,temperature,-0.0110,1.000000,False
6,Kendall-Tau,Lowest_Autumn_week,PCR_Postive_Count_lag_-6,temperature,0.2527,0.233112,False
7,Kendall-Tau,Lowest_Autumn_week,PCR_Postive_Count_lag_-7,temperature,0.1209,0.590640,False
8,Spearmann,Lowest_Autumn_week,PCR_Postive_Count,temperature,-0.1516,0.604791,False
9,Spearmann,Lowest_Autumn_week,PCR_Postive_Count_lag_-1,temperature,0.2835,0.325966,False


In [ ]:
#correlation for lagged data with current infected
data = low_aut_CrIn_lag.copy()
covid_param = ['Current_Infected','Current_Infected_lag_-1', 'Current_Infected_lag_-2','Current_Infected_lag_-3', 
               'Current_Infected_lag_-4','Current_Infected_lag_-5', 'Current_Infected_lag_-6',
               'Current_Infected_lag_-7']
weather_param = ['temperature']
final_list =[]
final_list = find_cor_sig(data,final_list,'Lowest_Autumn_week',covid_param,weather_param)

corr_aut_low_crin = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_aut_low_crin.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Autumn/Low_Week_CrIn_lag_Corr.csv')
corr_aut_low_crin

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Lowest_Autumn_week,Current_Infected,temperature,-0.0330,0.914485,False
1,Kendall-Tau,Lowest_Autumn_week,Current_Infected_lag_-1,temperature,-0.0769,0.747189,False
2,Kendall-Tau,Lowest_Autumn_week,Current_Infected_lag_-2,temperature,0.0989,0.667174,False
3,Kendall-Tau,Lowest_Autumn_week,Current_Infected_lag_-3,temperature,0.0110,1.000000,False
4,Kendall-Tau,Lowest_Autumn_week,Current_Infected_lag_-4,temperature,-0.0330,0.914485,False
5,Kendall-Tau,Lowest_Autumn_week,Current_Infected_lag_-5,temperature,-0.0330,0.914485,False
6,Kendall-Tau,Lowest_Autumn_week,Current_Infected_lag_-6,temperature,0.0549,0.829910,False
7,Kendall-Tau,Lowest_Autumn_week,Current_Infected_lag_-7,temperature,0.0110,1.000000,False
8,Spearmann,Lowest_Autumn_week,Current_Infected,temperature,-0.0286,0.922761,False
9,Spearmann,Lowest_Autumn_week,Current_Infected_lag_-1,temperature,-0.0154,0.958370,False


### Winter

In [ ]:
str_dt_win = '01-12-2021'
end_dt_win = '28-02-2022'
data_winter = Covid_Data_avg.loc[(Covid_Data_avg['Date'] >= pd.to_datetime(str_dt_win, format='%d-%m-%Y')) & (Covid_Data_avg['Date'] <= pd.to_datetime(end_dt_win, format='%d-%m-%Y'))]

data = data_winter
print(data[["temperature","relative_humidity"]].mean())

corr_kendall= kendalltau(data['temperature'],data['relative_humidity'])
print('Kendall-Tau Correlation between Temperature and Relative Humidity is',round(corr_kendall[0],4), 'for p-value', corr_kendall[1], 'and hence significance as',corr_kendall[1] <= 0.05)

corr_spear= spearmanr(data['temperature'],data['relative_humidity'])
print('Spearmann Correlation between Temperature and Relative Humidity is',round(corr_spear[0],4), 'for p-value', corr_spear[1], 'and hence significance as',corr_spear[1] <= 0.05) 

lags = 7 

for lag in range(1,lags+1):
    data_winter_PCR_lag = lagged_col_rev(data_winter, 'PCR_Postive_Count', lag, fill_value=0)
    
for lag in range(1,lags+1):
    data_winter_CrIn_lag = lagged_col_rev(data_winter, 'Current_Infected', lag, fill_value=0)
    
data_winter_PCR_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/Winter/data_winter_PCR_lag.csv')
data_winter_CrIn_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/Winter/data_winter_CrIn_lag.csv')




temperature           3.929539
relative_humidity    84.417926
dtype: float64
Kendall-Tau Correlation between Temperature and Relative Humidity is -0.1481 for p-value 0.038757323460147465 and hence significance as True
Spearmann Correlation between Temperature and Relative Humidity is -0.2094 for p-value 0.04756059036402346 and hence significance as True


In [ ]:
data = data_winter_PCR_lag.copy()
#print(data)
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="PCR_Postive_Count", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count_lag_-3'],name="PCR_Postive_Count lag 3", line=dict(color='green',
                              dash='dot')),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="relative_humidity", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Daily Positive Count", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green', range = [0,10000])

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
             #    showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
             #    zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered', range = [50,110])

fig.update_layout(plot_bgcolor="white")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    bordercolor="Black",
    borderwidth=1
))

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Winter/Lag_PCR.png")

In [ ]:
data = data_winter.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="Humidity", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside")

# Set y-axes titles
fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue', range = [-5,30])

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered', range = [30,100])

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Winter/Avg_Temp_Hum.png")

In [ ]:
data = data_winter.copy()
covid_param = ['PCR_Postive_Count','PCR_Positive_Hospitalised','Deceased', 'Current_Infected', 'Total_Confirmed']
weather_param = ['temperature','wind_speed','relative_humidity']
final_list =[]
final_list = find_cor_sig(data,final_list,'Winter',covid_param,weather_param)
corr_data = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_data.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Winter/Correlation.csv')
corr_data

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Winter,PCR_Postive_Count,temperature,0.0372,6.035498e-01,False
1,Kendall-Tau,Winter,PCR_Postive_Count,wind_speed,0.1905,7.831554e-03,True
2,Kendall-Tau,Winter,PCR_Postive_Count,relative_humidity,-0.3508,9.742057e-07,True
3,Kendall-Tau,Winter,PCR_Positive_Hospitalised,temperature,0.0843,2.414427e-01,False
4,Kendall-Tau,Winter,PCR_Positive_Hospitalised,wind_speed,0.0738,3.053798e-01,False
5,Kendall-Tau,Winter,PCR_Positive_Hospitalised,relative_humidity,-0.1771,1.384306e-02,True
6,Kendall-Tau,Winter,Deceased,temperature,0.0202,7.848027e-01,False
7,Kendall-Tau,Winter,Deceased,wind_speed,-0.0010,9.888273e-01,False
8,Kendall-Tau,Winter,Deceased,relative_humidity,0.0305,6.795337e-01,False
9,Kendall-Tau,Winter,Current_Infected,temperature,0.1251,8.079166e-02,False


In [ ]:
corr_data_pos = corr_data.loc[(corr_data["Significance"]==True)]
corr_data_pos

#Overall humidity has a very negligible correlation with daily count
#Temperature has a very negligible correlation with daily count but better than relative humidity

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
1,Kendall-Tau,Winter,PCR_Postive_Count,wind_speed,0.1905,7.831554e-03,True
2,Kendall-Tau,Winter,PCR_Postive_Count,relative_humidity,-0.3508,9.742057e-07,True
5,Kendall-Tau,Winter,PCR_Positive_Hospitalised,relative_humidity,-0.1771,1.384306e-02,True
10,Kendall-Tau,Winter,Current_Infected,wind_speed,0.1805,1.174092e-02,True
11,Kendall-Tau,Winter,Current_Infected,relative_humidity,-0.4577,1.675579e-10,True
13,Kendall-Tau,Winter,Total_Confirmed,wind_speed,0.2509,4.606062e-04,True
14,Kendall-Tau,Winter,Total_Confirmed,relative_humidity,-0.4871,1.048185e-11,True
16,Spearmann,Winter,PCR_Postive_Count,wind_speed,0.2986,4.256557e-03,True
17,Spearmann,Winter,PCR_Postive_Count,relative_humidity,-0.5240,1.157446e-07,True
20,Spearmann,Winter,PCR_Positive_Hospitalised,relative_humidity,-0.2563,1.476650e-02,True


In [ ]:
#Multiple Correlation
mul_list = []
weather_param = ['relative_humidity','wind_speed']
mul_list = mul_corr_calc(corr_data,data_winter,'Winter',weather_param,90)
corr_mul_win_TH = pd.DataFrame (mul_list, columns = ['CorrelationMethod','Season','Covid_Column','Relative_Humidity','Wind Speed','Multiple_Correlation'])
corr_mul_win_TH.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Winter/Multiple_Correlation.csv')
corr_mul_win_TH

,CorrelationMethod,Season,Covid_Column,Relative_Humidity,Wind Speed,Multiple_Correlation
0,Kendall-Tau,Winter,PCR_Postive_Count,-0.3508,0.1905,0.1099
1,Kendall-Tau,Winter,Current_Infected,-0.4577,0.1805,0.1959
2,Kendall-Tau,Winter,Total_Confirmed,-0.4871,0.2509,0.2205
3,Spearmann,Winter,PCR_Postive_Count,-0.5240,0.2986,0.2610
4,Spearmann,Winter,Current_Infected,-0.6460,0.2721,0.4052
5,Spearmann,Winter,Total_Confirmed,-0.6832,0.3711,0.4568


In [ ]:
data = data_winter.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="Daily Positive Count", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="Average Humidity [%]", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Daily Positive Count", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Winter/PCR_Hum.png")

In [ ]:
data = data_winter.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['Current_Infected'],name="Current Infected", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="Average Humidity [%]", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Current Infected", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Winter/CrIn_Hum.png")

In [ ]:
#correlation for lagged data for PCR count
data = data_winter_PCR_lag.copy()
covid_param = ['PCR_Postive_Count','PCR_Postive_Count_lag_-1', 'PCR_Postive_Count_lag_-2','PCR_Postive_Count_lag_-3', 
               'PCR_Postive_Count_lag_-4','PCR_Postive_Count_lag_-5', 'PCR_Postive_Count_lag_-6',
               'PCR_Postive_Count_lag_-7']
weather_param = ['relative_humidity']
final_list =[]
final_list = find_cor_sig(data,final_list,'Winter',covid_param,weather_param)

corr_data_temp = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_data_temp.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Winter/PCR_lag_Corr.csv')
corr_data_temp

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Winter,PCR_Postive_Count,relative_humidity,-0.3508,9.742057e-07,True
1,Kendall-Tau,Winter,PCR_Postive_Count_lag_-1,relative_humidity,-0.3728,1.955902e-07,True
2,Kendall-Tau,Winter,PCR_Postive_Count_lag_-2,relative_humidity,-0.3633,3.956455e-07,True
3,Kendall-Tau,Winter,PCR_Postive_Count_lag_-3,relative_humidity,-0.3728,1.955902e-07,True
4,Kendall-Tau,Winter,PCR_Postive_Count_lag_-4,relative_humidity,-0.3873,6.458273e-08,True
5,Kendall-Tau,Winter,PCR_Postive_Count_lag_-5,relative_humidity,-0.4032,1.815926e-08,True
6,Kendall-Tau,Winter,PCR_Postive_Count_lag_-6,relative_humidity,-0.3383,2.329449e-06,True
7,Kendall-Tau,Winter,PCR_Postive_Count_lag_-7,relative_humidity,-0.3653,3.415914e-07,True
8,Spearmann,Winter,PCR_Postive_Count,relative_humidity,-0.5240,1.157446e-07,True
9,Spearmann,Winter,PCR_Postive_Count_lag_-1,relative_humidity,-0.5491,2.100908e-08,True


In [ ]:
#correlation for lagged data for PCR count
data = data_winter_CrIn_lag.copy()
covid_param = ['Current_Infected','Current_Infected_lag_-1', 'Current_Infected_lag_-2','Current_Infected_lag_-3', 
               'Current_Infected_lag_-4','Current_Infected_lag_-5', 'Current_Infected_lag_-6',
               'Current_Infected_lag_-7']
weather_param = ['relative_humidity']
final_list =[]
final_list = find_cor_sig(data,final_list,'Winter',covid_param,weather_param)

corr_data_temp = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_data_temp.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Winter/CrIn_lag_Corr.csv')

### plotting it for the 14 days period with lag

In [ ]:
#Winter: 14.02.2022 to 27.02.2022
low_win = Covid_Data_avg.loc[(Covid_Data_avg['Date'] >= pd.to_datetime('14-02-2022', format='%d-%m-%Y')) & (Covid_Data_avg['Date'] <= pd.to_datetime('27-02-2022', format='%d-%m-%Y'))]
low_win= low_win.reset_index()



lags = 7 

for lag in range(1,lags+1):
    low_win_PCR_lag = lagged_col_rev(low_win, 'PCR_Postive_Count', lag, fill_value=0)
    
for lag in range(1,lags+1):
    low_win_CrIn_lag = lagged_col_rev(low_win, 'Current_Infected', lag, fill_value=0)

low_win_PCR_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/Winter/Lowest_week_PCR_lag.csv')
low_win_CrIn_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/Winter/Lowest_week_CrIn_lag.csv')


In [ ]:
data = low_win.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="Humidity", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside")

# Set y-axes titles
fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue', range = [-5,30])

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered', range = [30,100])

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Winter/Low_Week_Temp_Hum.png")

In [ ]:
data = low_win.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="Daily Positive Count", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="Average Humidity [%]", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Daily Positive Count", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Winter/Low_Week_PCR_Hum.png")

In [ ]:
data = low_win.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['Current_Infected'],name="Current Infected", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="Average Humidity [%]", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Current Infected", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Winter/Low_Week_CrIn_Hum.png")

In [ ]:
#correlation for lagged data with PCR count
data = low_win_PCR_lag.copy()
covid_param = ['PCR_Postive_Count','PCR_Postive_Count_lag_-1', 'PCR_Postive_Count_lag_-2','PCR_Postive_Count_lag_-3', 
               'PCR_Postive_Count_lag_-4','PCR_Postive_Count_lag_-5', 'PCR_Postive_Count_lag_-6',
               'PCR_Postive_Count_lag_-7']
weather_param = ['temperature']
final_list =[]
final_list = find_cor_sig(data,final_list,'Lowest_Winter_week',covid_param,weather_param)

corr_win_low_pcr = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_win_low_pcr.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Winter/Low_Week_PCR_lag_Corr.csv')
corr_win_low_pcr

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Lowest_Winter_week,PCR_Postive_Count,temperature,0.3626,0.079457,False
1,Kendall-Tau,Lowest_Winter_week,PCR_Postive_Count_lag_-1,temperature,0.0769,0.747189,False
2,Kendall-Tau,Lowest_Winter_week,PCR_Postive_Count_lag_-2,temperature,-0.1868,0.387988,False
3,Kendall-Tau,Lowest_Winter_week,PCR_Postive_Count_lag_-3,temperature,-0.5604,0.004566,True
4,Kendall-Tau,Lowest_Winter_week,PCR_Postive_Count_lag_-4,temperature,-0.3846,0.061658,False
5,Kendall-Tau,Lowest_Winter_week,PCR_Postive_Count_lag_-5,temperature,-0.2088,0.330811,False
6,Kendall-Tau,Lowest_Winter_week,PCR_Postive_Count_lag_-6,temperature,-0.2527,0.233112,False
7,Kendall-Tau,Lowest_Winter_week,PCR_Postive_Count_lag_-7,temperature,-0.4505,0.026400,True
8,Spearmann,Lowest_Winter_week,PCR_Postive_Count,temperature,0.4945,0.072250,False
9,Spearmann,Lowest_Winter_week,PCR_Postive_Count_lag_-1,temperature,0.2044,0.483346,False


In [ ]:
#correlation for lagged data with current infected
data = low_win_CrIn_lag.copy()
covid_param = ['Current_Infected','Current_Infected_lag_-1', 'Current_Infected_lag_-2','Current_Infected_lag_-3', 
               'Current_Infected_lag_-4','Current_Infected_lag_-5', 'Current_Infected_lag_-6',
               'Current_Infected_lag_-7']
weather_param = ['temperature']
final_list =[]
final_list = find_cor_sig(data,final_list,'Lowest_Winter_week',covid_param,weather_param)

corr_win_low_crin = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_win_low_crin.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Winter/Low_Week_CrIn_lag_Corr.csv')
corr_win_low_crin

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Lowest_Winter_week,Current_Infected,temperature,0.2747,0.192508,False
1,Kendall-Tau,Lowest_Winter_week,Current_Infected_lag_-1,temperature,0.2308,0.279180,False
2,Kendall-Tau,Lowest_Winter_week,Current_Infected_lag_-2,temperature,-0.1868,0.387988,False
3,Kendall-Tau,Lowest_Winter_week,Current_Infected_lag_-3,temperature,-0.4945,0.013838,True
4,Kendall-Tau,Lowest_Winter_week,Current_Infected_lag_-4,temperature,-0.5604,0.004566,True
5,Kendall-Tau,Lowest_Winter_week,Current_Infected_lag_-5,temperature,-0.3626,0.079457,False
6,Kendall-Tau,Lowest_Winter_week,Current_Infected_lag_-6,temperature,-0.2088,0.330811,False
7,Kendall-Tau,Lowest_Winter_week,Current_Infected_lag_-7,temperature,-0.3846,0.061658,False
8,Spearmann,Lowest_Winter_week,Current_Infected,temperature,0.3714,0.191021,False
9,Spearmann,Lowest_Winter_week,Current_Infected_lag_-1,temperature,0.2703,0.349919,False


### Spring 

In [ ]:
str_dt_spr = '01-03-2022'
end_dt_spr = '31-05-2022'
data_spring = Covid_Data_avg.loc[(Covid_Data_avg['Date'] >= pd.to_datetime(str_dt_spr, format='%d-%m-%Y')) & (Covid_Data_avg['Date'] <= pd.to_datetime(end_dt_spr, format='%d-%m-%Y'))]


data = data_spring
print(data[["temperature","relative_humidity"]].mean())

corr_kendall= kendalltau(data['temperature'],data['relative_humidity'])
print('Kendall-Tau Correlation between Temperature and Relative Humidity is',round(corr_kendall[0],4), 'for p-value', corr_kendall[1], 'and hence significance as',corr_kendall[1] <= 0.05)

corr_spear= spearmanr(data['temperature'],data['relative_humidity'])
print('Spearmann Correlation between Temperature and Relative Humidity is',round(corr_spear[0],4), 'for p-value', corr_spear[1], 'and hence significance as',corr_spear[1] <= 0.05) 

lags = 7 

for lag in range(1,lags+1):
    data_spring_PCR_lag = lagged_col_rev(data_spring, 'PCR_Postive_Count', lag, fill_value=0)
    
for lag in range(1,lags+1):
    data_spring_CrIn_lag = lagged_col_rev(data_spring, 'Current_Infected', lag, fill_value=0)
    
data_spring_PCR_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/Spring/data_spring_PCR_lag.csv')
data_spring_CrIn_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/Spring/data_spring_CrIn_lag.csv')

temperature           9.946133
relative_humidity    63.351967
dtype: float64
Kendall-Tau Correlation between Temperature and Relative Humidity is -0.0025 for p-value 0.9739718372175088 and hence significance as False
Spearmann Correlation between Temperature and Relative Humidity is 0.0008 for p-value 0.9940929654012818 and hence significance as False


In [ ]:
data = data_spring.copy()
covid_param = ['PCR_Postive_Count','PCR_Positive_Hospitalised','Deceased', 'Current_Infected', 'Total_Confirmed']
weather_param = ['temperature','wind_speed','relative_humidity']
final_list =[]
final_list = find_cor_sig(data,final_list,'Spring',covid_param,weather_param)
corr_data = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_data.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Spring/Correlation.csv')
corr_data

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Spring,PCR_Postive_Count,temperature,-0.1951,9.949030e-03,True
1,Kendall-Tau,Spring,PCR_Postive_Count,wind_speed,-0.1173,1.211847e-01,False
2,Kendall-Tau,Spring,PCR_Postive_Count,relative_humidity,-0.0093,9.026185e-01,False
3,Kendall-Tau,Spring,PCR_Positive_Hospitalised,temperature,-0.0851,2.619787e-01,False
4,Kendall-Tau,Spring,PCR_Positive_Hospitalised,wind_speed,-0.0801,2.907570e-01,False
5,Kendall-Tau,Spring,PCR_Positive_Hospitalised,relative_humidity,0.0727,3.377772e-01,False
6,Kendall-Tau,Spring,Deceased,temperature,0.0382,6.233791e-01,False
7,Kendall-Tau,Spring,Deceased,wind_speed,-0.0248,7.495798e-01,False
8,Kendall-Tau,Spring,Deceased,relative_humidity,0.0700,3.679662e-01,False
9,Kendall-Tau,Spring,Current_Infected,temperature,-0.3506,3.602830e-06,True


In [ ]:
#Multiple Correlation
mul_list = []
weather_param = ['temperature','relative_humidity']
mul_list = mul_corr_calc(corr_data,data_spring,'Spring',weather_param,90)
corr_mul_spr_TH = pd.DataFrame (mul_list, columns = ['CorrelationMethod','Season','Covid_Column','Temperature','Relative_Humidity','Multiple_Correlation'])
corr_mul_spr_TH.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Spring/Multiple_Correlation.csv')
corr_mul_spr_TH

,CorrelationMethod,Season,Covid_Column,Temperature,Relative_Humidity,Multiple_Correlation
0,Spearmann,Spring,Total_Confirmed,0.8174,0.2584,0.709


In [ ]:
data = data_spring.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="Humidity", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside")

# Set y-axes titles
fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue', range = [-5,30])

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered', range = [30,100])

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Spring/Avg_Temp_Hum.png")

In [ ]:
data = data_spring.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="Daily Positive Count", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Daily Positive Count", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Spring/PCR_Temp.png")

In [ ]:
data = data_spring.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['Current_Infected'],name="Current Infected", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Current Infected", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Spring/CrIn_Temp.png")

In [ ]:
#correlation for lagged data with PCR count
data = data_spring_PCR_lag.copy()
covid_param = ['PCR_Postive_Count','PCR_Postive_Count_lag_-1', 'PCR_Postive_Count_lag_-2','PCR_Postive_Count_lag_-3', 
               'PCR_Postive_Count_lag_-4','PCR_Postive_Count_lag_-5', 'PCR_Postive_Count_lag_-6',
               'PCR_Postive_Count_lag_-7']
weather_param = ['temperature']
final_list =[]
final_list = find_cor_sig(data,final_list,'Spring',covid_param,weather_param)

corr_data_temp = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_data_temp.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Spring/PCR_Lag_Correlation.csv')
corr_data_temp
#no concrete lag correlation

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Spring,PCR_Postive_Count,temperature,-0.1951,0.009949,True
1,Kendall-Tau,Spring,PCR_Postive_Count_lag_-1,temperature,-0.1914,0.011450,True
2,Kendall-Tau,Spring,PCR_Postive_Count_lag_-2,temperature,-0.2433,0.001310,True
3,Kendall-Tau,Spring,PCR_Postive_Count_lag_-3,temperature,-0.2781,0.000238,True
4,Kendall-Tau,Spring,PCR_Postive_Count_lag_-4,temperature,-0.2475,0.001072,True
5,Kendall-Tau,Spring,PCR_Postive_Count_lag_-5,temperature,-0.2154,0.004417,True
6,Kendall-Tau,Spring,PCR_Postive_Count_lag_-6,temperature,-0.1463,0.053216,False
7,Kendall-Tau,Spring,PCR_Postive_Count_lag_-7,temperature,-0.0611,0.419364,False
8,Spearmann,Spring,PCR_Postive_Count,temperature,-0.2981,0.006869,True
9,Spearmann,Spring,PCR_Postive_Count_lag_-1,temperature,-0.2810,0.011043,True


In [ ]:
data = data_spring_PCR_lag.copy()
#print(data)
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="PCR_Postive_Count", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count_lag_-3'],name="PCR_Postive_Count lag 3", line=dict(color='green',
                              dash='dot')),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="temperature", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Daily Positive Count", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=1, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green', range = [0,17000])

fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
              #   showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
              #   zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue', range = [0,25])

fig.update_layout(plot_bgcolor="white")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    bordercolor="Black",
    borderwidth=1
))

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Spring/Lag_PCR.png")

In [ ]:
#correlation for lagged data with PCR count
data = data_spring_CrIn_lag.copy()
covid_param = ['Current_Infected','Current_Infected_lag_-1', 'Current_Infected_lag_-2','Current_Infected_lag_-3', 
               'Current_Infected_lag_-4','Current_Infected_lag_-5', 'Current_Infected_lag_-6',
               'Current_Infected_lag_-7']
weather_param = ['temperature']
final_list =[]
final_list = find_cor_sig(data,final_list,'Spring',covid_param,weather_param)

corr_data_temp = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_data_temp.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Spring/CrIn_Lag_Correlation.csv')
corr_data_temp
#no concrete lag correlation

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Spring,Current_Infected,temperature,-0.3506,3.602830e-06,True
1,Kendall-Tau,Spring,Current_Infected_lag_-1,temperature,-0.3247,1.782844e-05,True
2,Kendall-Tau,Spring,Current_Infected_lag_-2,temperature,-0.3272,1.538443e-05,True
3,Kendall-Tau,Spring,Current_Infected_lag_-3,temperature,-0.3241,1.849490e-05,True
4,Kendall-Tau,Spring,Current_Infected_lag_-4,temperature,-0.3025,6.419409e-05,True
5,Kendall-Tau,Spring,Current_Infected_lag_-5,temperature,-0.2840,1.753357e-04,True
6,Kendall-Tau,Spring,Current_Infected_lag_-6,temperature,-0.2500,9.548280e-04,True
7,Kendall-Tau,Spring,Current_Infected_lag_-7,temperature,-0.1926,1.093010e-02,True
8,Spearmann,Spring,Current_Infected,temperature,-0.5275,4.204271e-07,True
9,Spearmann,Spring,Current_Infected_lag_-1,temperature,-0.4942,2.744425e-06,True


### plotting it for the 14 days period with lag 

In [ ]:
#Spring: 28.02.2022 to 13.03.2022
low_spr = Covid_Data_avg.loc[(Covid_Data_avg['Date'] >= pd.to_datetime('28-02-2022', format='%d-%m-%Y')) & (Covid_Data_avg['Date'] <= pd.to_datetime('13-03-2022', format='%d-%m-%Y'))]
low_spr= low_spr.reset_index()
#low_spr

lags = 7 
for lag in range(1,lags+1):
    low_spr_PCR_lag = lagged_col_rev(low_spr, 'PCR_Postive_Count', lag, fill_value=0)
    
for lag in range(1,lags+1):
    low_spr_CrIn_lag = lagged_col_rev(low_spr, 'Current_Infected', lag, fill_value=0)

low_spr_PCR_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/Spring/Low_Week_PCR_lag.csv')
low_spr_CrIn_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/Spring/Low_Week_CrIn_lag.csv')


In [ ]:
data = low_spr.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="Humidity", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", dtick="M1", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue', range = [-5,30])

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered', range = [30,100])

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Spring/Low_Week_Temp_Hum.png")


In [ ]:
data = low_spr.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="Daily Positive Count", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Daily Positive Count", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Spring/Low_Week_PCR_Temp.png")

In [ ]:
data = low_spr.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['Current_Infected'],name="Current Infected", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Current Infected", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Spring/Low_Week_CrIn_Temp.png")

In [ ]:
#correlation for lagged data with PCR count
data = low_spr_PCR_lag.copy()
covid_param = ['PCR_Postive_Count','PCR_Postive_Count_lag_-1', 'PCR_Postive_Count_lag_-2','PCR_Postive_Count_lag_-3', 
               'PCR_Postive_Count_lag_-4','PCR_Postive_Count_lag_-5', 'PCR_Postive_Count_lag_-6',
               'PCR_Postive_Count_lag_-7']
weather_param = ['temperature']
final_list =[]
final_list = find_cor_sig(data,final_list,'Lowest_Spring_week',covid_param,weather_param)

corr_spr_low_pcr = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
low_spr_CrIn_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Spring/Low_Week_PCR_lag_Corr.csv')

#correlation for lagged data with current infected
data = low_spr_CrIn_lag.copy()
covid_param = ['Current_Infected','Current_Infected_lag_-1', 'Current_Infected_lag_-2','Current_Infected_lag_-3', 
               'Current_Infected_lag_-4','Current_Infected_lag_-5', 'Current_Infected_lag_-6',
               'Current_Infected_lag_-7']
weather_param = ['temperature']
final_list =[]
final_list = find_cor_sig(data,final_list,'Lowest_Spring_week',covid_param,weather_param)

corr_spr_low_crin = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_spr_low_crin.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Spring/Low_Week_CrIn_lag_Corr.csv')

### Summer 

In [ ]:
str_dt_sum = '02-06-2022'
data_summer = Covid_Data_avg.loc[Covid_Data_avg['Date'] >= pd.to_datetime(str_dt_sum, format='%d-%m-%Y')]

lags = 7 

for lag in range(1,lags+1):
    data_summer_PCR_lag = lagged_col_rev(data_summer, 'PCR_Postive_Count', lag, fill_value=0)
    
for lag in range(1,lags+1):
    data_summer_CrIn_lag = lagged_col_rev(data_summer, 'Current_Infected', lag, fill_value=0)

data_summer_PCR_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/Summer/data_summer_PCR_lag.csv')
data_summer_CrIn_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/Summer/data_summer_CrIn_lag.csv')

data = data_summer
print(data[["temperature","relative_humidity"]].mean())

corr_kendall= kendalltau(data['temperature'],data['relative_humidity'])
print('Kendall-Tau Correlation between Temperature and Relative Humidity is',round(corr_kendall[0],4), 'for p-value', corr_kendall[1], 'and hence significance as',corr_kendall[1] <= 0.05)

corr_spear= spearmanr(data['temperature'],data['relative_humidity'])
print('Spearmann Correlation between Temperature and Relative Humidity is',round(corr_spear[0],4), 'for p-value', corr_spear[1], 'and hence significance as',corr_spear[1] <= 0.05) 

data = data_summer_PCR_lag.copy()
#print(data)
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="PCR_Postive_Count", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count_lag_-3'],name="PCR_Postive_Count lag 3", line=dict(color='green',
                              dash='dot')),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="temperature", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Summer/Lag_PCR.png")


temperature          19.556126
relative_humidity    60.035751
dtype: float64
Kendall-Tau Correlation between Temperature and Relative Humidity is -0.4179 for p-value 0.00014572879126074312 and hence significance as True
Spearmann Correlation between Temperature and Relative Humidity is -0.5812 for p-value 8.393068142663371e-05 and hence significance as True


In [ ]:
data = data_summer.copy()
covid_param = ['PCR_Postive_Count','PCR_Positive_Hospitalised','Deceased', 'Current_Infected', 'Total_Confirmed']
weather_param = ['temperature','wind_speed','relative_humidity']
final_list =[]
final_list = find_cor_sig(data,final_list,'Summer',covid_param,weather_param)
corr_data = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_data.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Summer/Correlation.csv')

#Multiple Correlation
mul_list = []
weather_param = ['temperature','relative_humidity']
mul_list = mul_corr_calc(corr_data,data_summer,'Summer',weather_param,40)
corr_mul_sum_TH = pd.DataFrame (mul_list, columns = ['CorrelationMethod','Season','Covid_Column','Temperature','Relative_Humidity','Multiple_Correlation'])
corr_mul_sum_TH.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Summer/Multiple_Correlation.csv')
corr_mul_sum_TH

data = data_summer.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="Humidity", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", dtick="M1", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue', range = [-5,30])

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered', range = [30,100])

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Summer/Avg_Temp_Hum.png")

data = data_summer.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="Daily Positive Count", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Daily Positive Count", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Summer/PCR_Temp.png")

data = data_summer.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['Current_Infected'],name="Current Infected", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Current Infected", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Summer/CrIn_Temp.png")

#correlation for lagged data with PCR count
data = data_summer_PCR_lag.copy()
covid_param = ['PCR_Postive_Count','PCR_Postive_Count_lag_-1', 'PCR_Postive_Count_lag_-2','PCR_Postive_Count_lag_-3', 
               'PCR_Postive_Count_lag_-4','PCR_Postive_Count_lag_-5', 'PCR_Postive_Count_lag_-6',
               'PCR_Postive_Count_lag_-7']
weather_param = ['temperature']
final_list =[]
final_list = find_cor_sig(data,final_list,'Summer',covid_param,weather_param)

corr_data_temp = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_data_temp.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Summer/PCR_lag_Correlation.csv')

#correlation for lagged data with PCR count
data = data_summer_CrIn_lag.copy()
covid_param = ['Current_Infected','Current_Infected_lag_-1', 'Current_Infected_lag_-2','Current_Infected_lag_-3', 
               'Current_Infected_lag_-4','Current_Infected_lag_-5', 'Current_Infected_lag_-6',
               'Current_Infected_lag_-7']
weather_param = ['temperature']
final_list =[]
final_list = find_cor_sig(data,final_list,'Summer',covid_param,weather_param)

corr_data_temp = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_data_temp.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Summer/CrIn_lag_Correlation.csv')

### plotting it for the 14 days period with lag 

In [ ]:
#Summer: 04.06.2022 to 17.06.2022
low_sum = Covid_Data_avg.loc[(Covid_Data_avg['Date'] >= pd.to_datetime('04-06-2022', format='%d-%m-%Y')) & (Covid_Data_avg['Date'] <= pd.to_datetime('17-06-2022', format='%d-%m-%Y'))]
low_sum= low_sum.reset_index()

lags = 7 

for lag in range(1,lags+1):
    low_sum_PCR_lag = lagged_col_rev(low_sum, 'PCR_Postive_Count', lag, fill_value=0)
    
for lag in range(1,lags+1):
    low_sum_CrIn_lag = lagged_col_rev(low_sum, 'Current_Infected', lag, fill_value=0)

low_sum_PCR_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/Summer/Low_Week_PCR_lag.csv')
low_sum_CrIn_lag.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/datasets/Summer/Low_Week_CrIn_lag.csv')

data = low_sum.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="Humidity", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", dtick="M1", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue', range = [-5,30])

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered', range = [30,100])

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Summer/Low_Week_Temp_Hum.png")

data = low_sum.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="Daily Positive Count", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Daily Positive Count", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Summer/Low_Week_PCR_Temp.png")

data = low_sum.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['Current_Infected'],name="Current Infected", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Current Infected", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Summer/Low_Week_CrIn_Temp.png")

#correlation for lagged data with PCR count
data = low_sum_PCR_lag.copy()
covid_param = ['PCR_Postive_Count','PCR_Postive_Count_lag_-1', 'PCR_Postive_Count_lag_-2','PCR_Postive_Count_lag_-3', 
               'PCR_Postive_Count_lag_-4','PCR_Postive_Count_lag_-5', 'PCR_Postive_Count_lag_-6',
               'PCR_Postive_Count_lag_-7']
weather_param = ['temperature']
final_list =[]
final_list = find_cor_sig(data,final_list,'Lowest_Summer_week',covid_param,weather_param)

corr_sum_low_pcr = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_sum_low_pcr.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Summer/Low_Week_PCR_lag_Corr.csv')

#correlation for lagged data with current infected
data = low_sum_CrIn_lag.copy()
covid_param = ['Current_Infected','Current_Infected_lag_-1', 'Current_Infected_lag_-2','Current_Infected_lag_-3', 
               'Current_Infected_lag_-4','Current_Infected_lag_-5', 'Current_Infected_lag_-6',
               'Current_Infected_lag_-7']
weather_param = ['temperature']
final_list =[]
final_list = find_cor_sig(data,final_list,'Lowest_Summer_week',covid_param,weather_param)

corr_sum_low_crin = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
corr_sum_low_crin.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/Summer/Low_Week_CrIn_lag_Corr.csv')